In [2]:
# 추천 API (Flask 기반)
from flask import Flask, request, jsonify
import joblib
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer

app = Flask(__name__)

# 1. 모델 및 벡터라이저 로딩
df = pd.read_excel('/content/drive/MyDrive/졸업논문/강원도_관광지_20_예시.xlsx')
tfidf_vectorizer = joblib.load('/content/drive/MyDrive/졸업논문/tfidf_vectorizer.pkl')

models = {
    'season': joblib.load('/content/drive/MyDrive/졸업논문/season_model.pkl'),
    'nature': joblib.load('/content/drive/MyDrive/졸업논문/nature_model.pkl'),
    'vibe': joblib.load('/content/drive/MyDrive/졸업논문/vibe_model.pkl'),
    'target': joblib.load('/content/drive/MyDrive/졸업논문/target_model.pkl')
}
encoders = {
    'season': joblib.load('/content/drive/MyDrive/졸업논문/season_encoder.pkl'),
    'nature': joblib.load('/content/drive/MyDrive/졸업논문/nature_encoder.pkl'),
    'vibe': joblib.load('/content/drive/MyDrive/졸업논문/vibe_encoder.pkl'),
    'target': joblib.load('/content/drive/MyDrive/졸업논문/target_encoder.pkl')
}

# 전처리
df['season'] = df['season'].astype(str)
df['nature'] = df['nature'].apply(lambda x: [t.strip() for t in str(x).split(',')])
df['vibe'] = df['vibe'].apply(lambda x: [t.strip() for t in str(x).split(',')])
df['target'] = df['target'].apply(lambda x: [t.strip() for t in str(x).split(',')])
X_vec = tfidf_vectorizer.transform(df['description'])

@app.route("/recommend", methods=["POST"])
def recommend_places():
    user_input = request.json  # 안드로이드에서 JSON으로 POST된 데이터

    weights = {
        'season': 1.0,
        'nature': 1.0,
        'vibe': 1.0,
        'target': 1.0
    }
    total_scores = np.zeros(X_vec.shape[0])

    for key in ['season', 'nature', 'vibe', 'target']:
        model = models[key]
        encoder = encoders[key]

        if key in ['nature', 'vibe', 'target']:
            user_tags = user_input.get(key, [])
            tag_indices = [i for i, tag in enumerate(encoder.classes_) if tag in user_tags]
            if tag_indices:
                probas = np.sum(model.predict_proba(X_vec)[:, tag_indices], axis=1)
            else:
                probas = np.zeros(X_vec.shape[0])
        else:
            label_index = encoder.transform([user_input[key]])[0]
            probas = model.predict_proba(X_vec)[:, label_index]

        total_scores += weights[key] * probas

    # 결과 정렬 및 추천 사유 작성
    df['추천점수'] = total_scores
    recommendations = df.sort_values(by='추천점수', ascending=False).head(3).copy()

    explanation = (
        f"사용자가 선택한 조건 "
        f"'{user_input.get('season')}', "
        f"{', '.join(user_input.get('nature', []))}, "
        f"{', '.join(user_input.get('vibe', []))}, "
        f"{', '.join(user_input.get('target', []))} "
        f"에 따라 추천되었습니다."
    )

    results = []
    for _, row in recommendations.iterrows():
        results.append({
            "name": row["name"],
            "city": row["city"],
            "description": row["description"],
            "score": round(row["추천점수"], 2),
            "reason": explanation
        })

    return jsonify(results)

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000)


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/졸업논문/강원도_관광지_20_예시.xlsx'